In [4]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import json
import re
import requests
 
#네이버 검색 Open API 사용 요청시 얻게되는 정보를 입력합니다
naver_client_id = 'PbDkBrbwVr7E0kjp_eBN'
naver_client_secret = 'SXuLVGIhRh'
 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
 


In [71]:
def searchByTitle(title):
    myurl = 'https://openapi.naver.com/v1/search/movie.json?display=100&query=' + quote(title)
    request = urllib.request.Request(myurl)
    request.add_header("X-Naver-Client-Id",naver_client_id)
    request.add_header("X-Naver-Client-Secret",naver_client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        d = json.loads(response_body.decode('utf-8'))
        if (len(d['items']) > 0):
            return d['items']
        else:
            return None
 
    else:
        print("Error Code:" + rescode)

In [81]:
def findItemByInput(items):
    for index, item in enumerate(items):
        navertitle = cleanhtml(item['title'])
        naversubtitle = cleanhtml(item['subtitle'])
        naverpubdate = cleanhtml(item['pubDate'])
        naveractor = cleanhtml(item['actor'])
        naverlink = cleanhtml(item['link'])
        naveruserScore = cleanhtml(item['userRating'])
 
        navertitle1 = navertitle.replace(" ","")
        navertitle1 = navertitle1.replace("-", ",")
        navertitle1 = navertitle1.replace(":", ",")
 
        #기자 평론가 평점을 얻어 옵니다
        spScore = getSpecialScore(naverlink)
 
        #네이버가 다루는 영화 고유 ID를 얻어 옵니다다
        naverid = re.split("code=", naverlink)[1]
 
        # 영화의 타이틀 이미지를 표시합니다
        # if (item['image'] != None and "http" in item['image']):
        #    response = requests.get(item['image'])
        #    img = Image.open(BytesIO(response.content))
        #    img.show()
 
        print(index, navertitle, naversubtitle, naveruserScore, spScore)

In [73]:
def getInfoFromNaver(searchTitle):
    items = searchByTitle(searchTitle)
 
    if (items != None):
        findItemByInput(items)
    else:
        print("No result")


In [74]:
def get_soup(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    return soup
 


In [75]:
#기자 평론가 평점을 얻어 옵니다
def getSpecialScore(URL):
    soup = get_soup(URL)
    scorearea = soup.find_all('div', "spc_score_area")
    newsoup = BeautifulSoup(str(scorearea), 'lxml')
    score = newsoup.find_all('em')
    if (score and len(score) > 5):
        scoreis = score[1].text + score[2].text + score[3].text + score[4].text
        return float(scoreis)
    else:
        return 0.0
 

In [76]:
import pandas as pd

In [82]:
import time
from selenium import webdriver
import xlrd
from xlutils.copy import copy
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import os
new_list = []
for i in range(0,5):
# 엑셀 파일 열기
    wb = xlrd.open_workbook('KOBIS_300.xlsx')
    sheet = wb.sheet_by_index(0)
    copy_wb = copy(wb)
    copy_sheet = copy_wb.get_sheet(0)
# 검색 키워드 셀에서 가져오기
    name = sheet.cell(i+1, 1).value
    keyword = str(name) #숫자인 경우가 있어서 str()

#getInfoFromNaver(u"더 킹")
    #getInfoFromNaver(keyword)
    
    new_list.append(getInfoFromNaver(keyword))
   # df = pd.DataFrame(getInfoFromNaver(keyword))  
    

0 택시운전사 A Taxi Driver 9.04 6.09
1 택시 운전사의 사랑 Midnight My Love 8.56 0.0
2 택시 운전사의 사랑 The Cabbie 5.50 0.0
0 신과함께-죄와 벌 Along With the Gods: The Two Worlds 7.83 5.92
0 공조 Confidential Assignment 8.27 5.0
0 스파이더맨: 파 프롬 홈 Spider-Man: Far From Home 9.24 0.0
1 스파이더맨: 홈커밍 Spider-Man: Homecoming 8.37 7.0
0 대범죄도시: 나쁜녀석들 The Scoundrels 0.00 0.0
1 범죄도시 THE OUTLAWS 9.14 6.0
2 바이스: 범죄도시 Vice 4.18 0.0
3 줄루 범죄도시 Zulu 7.56 0.0
4 범죄도시 What Doesn't Kill You 7.70 0.0


In [83]:
new_list

[None, None, None, None, None]